In [1]:
import importlib
import numpy as np
import pandas as pd
import utils
from glob import glob

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm_statefull/utils.py'>

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Bidirectional
from keras.preprocessing.sequence import pad_sequences
# from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
word2ind, ind2word = utils.load_index_word_map()

In [5]:
vocab_size = len(word2ind)

In [6]:
batch_size = 0

while batch_size < 500:
    train_filename = np.random.choice(glob('data/train_*.csv'), size=1)[0]
    before_scan = pd.read_csv(train_filename)
    batch_size = before_scan.shape[0]
batch_size

667

In [7]:
def scanning(array, input_length):
    batches = []
    text_length = array.shape[1]
    for i in range(text_length-input_length):
        batches.append(array[:, i:i+input_length+1])
    return np.concatenate(batches)

In [8]:
emb_dim = 128
input_length = 300

In [9]:
dataset = scanning(before_scan.values, input_length)

In [10]:
dataset.shape

(411539, 301)

In [11]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(batch_size, input_length)))
model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Dense(vocab_size, activation='softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (667, 300, 128)           12800     
_________________________________________________________________
conv1d_1 (Conv1D)            (667, 300, 256)           131328    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (667, 150, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)          (667, 150, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (667, 150, 32)            32800     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (667, 75, 32)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (667, 75, 200)            106400    
__________

In [13]:
model.set_weights(load_model('model_300.h5').get_weights())

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
for _ in range(2):
    print(_)
    model.reset_states()
    model_hist = model.fit(dataset[:, :-1], dataset[:, [-1]], epochs=1, batch_size=batch_size)
    if model_hist.history['acc'][0] > .55: break

0
Epoch 1/1
411539/411539 [==============================] - 206s - loss: 1.8179 - acc: 0.4702   
1
Epoch 1/1
411539/411539 [==============================] - 204s - loss: 1.7303 - acc: 0.4903   


In [16]:
# save model
model.save('model_300.h5')

In [17]:
pred_model = Sequential()
pred_model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(1, input_length)))
pred_model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Dense(vocab_size, activation='softmax'))

In [18]:
pred_model.set_weights(model.get_weights())

In [19]:
def implement(seed_text, maxlen=100, must_stop=2000, n_likely=5):
    cleaned = list(seed_text)
    padded_input_tokens = utils.tokenise(word2ind, cleaned)
    res_tokens = [token for token in padded_input_tokens]
    pred_model.reset_states()
    while must_stop > 0:
        padded_input_tokens = pad_sequences([padded_input_tokens], maxlen=maxlen)
        probs = pred_model.predict(padded_input_tokens, batch_size=1)[0]
        first_n = np.argsort(probs)[-n_likely:]
        probs = probs[first_n] / np.sum(probs[first_n])
        predicted = np.random.choice(first_n, p=probs)
        padded_input_tokens = padded_input_tokens[0].tolist()
        padded_input_tokens.append(predicted)
        res_tokens.append(predicted)
        must_stop -= 1
    detokenised = utils.detokenise(ind2word, res_tokens)
    return ''.join(detokenised)

In [20]:
res = implement('We are friends, good friends', 300, 800, 2)

In [21]:
print(res)

We are friends, good friends the world the sear
I wanna so to the song
I know I'm so the was the world of the was
I can't loner in the song on the see
And I can the see you
I'm not in love in you
I'm gonna word of my live
And I know I'm never like the see you can't see
And I like a see
I know you can't let
I can the wait a like the way it to love
I know your set it
It's the ways a pay it see
I could star on
The world tomora
One mare
I'm not in long in you
I'm not in love in my long
I gone  
And I've got in the was the wait the world
I kneus a sean the sear to
I know
The words on your soul the sean
I've got to the would the wait
I get in your somewe to like a sear to that's to love
I'm not in love  
I'm not in love it no love  
I'm not in love in my love
I'm not in love in you don't love in you
I'm not in love you kno
